In [14]:
''' FOR DEVELOPMENT ONLY, DELETE CELL '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2

# data array processing
import numpy as np
#import pandas as pd

#from scipy.stats import linregress
#from scipy.interpolate import splev, splrep

#from scipy.signal import savgol_filter

#from scipy.optimize import curve_fit


# plotting with matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

#import matplotlib.colors as colors
#import matplotlib.cm as cmx

import seaborn as sns

## development override params
pvlibs_path = '/home/brendan/dev/pvlibs'
base_path = '../data/iv/wavelabs'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
''' initialise notebook '''

# add location to path
#pvlibs_path = '/srv/pvlibs'
import sys
sys.path.append(pvlibs_path)

# plotting with matplotlib, set interactive figure display
%matplotlib widget

# import pvlibs package
import pvlibs


In [26]:
''' import measurement files '''

# define directory to search; "./" for current directory, "../" for up one directory
#base_path = './data'

# define measurement properties (!required!)
props = {
    # measurement type
    'meas_type': 'iv', # 'slt' for sinton lifetime
    'file_ext': 'xlsx', # ['prm'|'xlsx']
    'file_type': 'wavelabs', # ['halm-500'|'wavelabs']
    
    # wafer properties
    'wafer_area': .243, # [cm^-2]
    
    # measurement conditions
    'temperature': 298.15,
}

# generate measurement file database
db = pvlibs.init_file_db(base_path, props)


begin file search and import 

imported reference to file: multi-1-back-std.xlsx
imported reference to file: mono-14-back-std.xlsx
imported reference to file: multi-1-front-std.xlsx
imported reference to file: mono-16-front-std.xlsx
imported reference to file: mono-13-front-std.xlsx
imported reference to file: mono-13-back-std.xlsx
imported reference to file: mono-14-front-std.xlsx
imported reference to file: multi-2-back-std.xlsx
imported reference to file: mono-12-back-std.xlsx
imported reference to file: mono-16-back-std.xlsx
imported reference to file: mono-15-front-std.xlsx
imported reference to file: mono-15-back-std.xlsx
imported reference to file: mono-12-front-std.xlsx
imported reference to file: multi-2-front-std.xlsx

file search and import complete

14 measurement file references imported


In [27]:
''' parse parameters from file name '''

# define single parameter separator; e.g. '-', '_', or '\s' (space)
param_sep = '-'

# define ordered list of parameters within filename separated by param_sep
params = [
    'wafer',
    'device',
    'side',
    'polarity',
]

# parse file name parameters from each measurement file
db = pvlibs.parse_file_names(db, param_sep, params)


begin file name parameter parsing 

parsing: file 1/14
parsing: file 2/14
parsing: file 3/14
parsing: file 4/14
parsing: file 5/14
parsing: file 6/14
parsing: file 7/14
parsing: file 8/14
parsing: file 9/14
parsing: file 10/14
parsing: file 11/14
parsing: file 12/14
parsing: file 13/14
parsing: file 14/14

file name parameter parsing complete

14 file names parsed


In [28]:
set([n['device'] for n in db])

{'1', '12', '13', '14', '15', '16', '2'}

In [29]:
db[0]

{'meas_type': 'iv',
 'file_ext': 'xlsx',
 'file_type': 'wavelabs',
 'wafer_area': 0.243,
 'temperature': 298.15,
 'file_name': 'multi-1-back-std.xlsx',
 'file_path': '../data/iv/wavelabs',
 'wafer': 'multi',
 'device': '1',
 'side': 'back',
 'polarity': 'std'}

In [32]:

node = db[0]

pvlibs.data_import.core.import_data_file(node['meas_type'], node['file_type'], node['file_path'], node['file_name'])

{'full': {'voltage': array([ 0.6500104 ,  0.64933544,  0.64798546,  0.64663553,  0.6459605 ,
          0.6446106 ,  0.64393556,  0.6425856 ,  0.64123565,  0.6405606 ,
          0.6392107 ,  0.6385357 ,  0.6365107 ,  0.63583577,  0.6338108 ,
          0.63246083,  0.63111085,  0.62976086,  0.6277359 ,  0.62706095,
          0.625036  ,  0.62301105,  0.62166107,  0.6196361 ,  0.61693615,
          0.6149112 ,  0.61288625,  0.6108613 ,  0.6081614 ,  0.6054614 ,
          0.6020865 ,  0.5987116 ,  0.5953367 ,  0.5912868 ,  0.5872368 ,
          0.58183694,  0.57643706,  0.56968725,  0.5615874 ,  0.5507877 ,
          0.53458804,  0.5089386 ,  0.46236464,  0.39081624,  0.30981806,
          0.22949485,  0.14984664,  0.07087341, -0.00742483, -0.08639807,
         -0.16672127, -0.24704446, -0.32669267, -0.40769085, -0.48801407,
         -0.56833726, -0.64798546, -0.7303336 , -0.8099818 , -0.891655  ,
         -0.97062826, -1.0516264 , -1.1326246 , -1.2142978 , -1.293946  ,
         -1.3762941

In [33]:
''' import measurement data '''

# import data from files
db = pvlibs.import_file_data(db)


begin file data import 

importing: file 1/14
importing: file 2/14
importing: file 3/14
importing: file 4/14
importing: file 5/14
importing: file 6/14
importing: file 7/14
importing: file 8/14
importing: file 9/14
importing: file 10/14
importing: file 11/14
importing: file 12/14
importing: file 13/14
importing: file 14/14

data file import complete

data imported from 14 files


In [34]:

fig = plt.figure()
ax = []; ax.append( fig.add_subplot() )#; ax.append( ax[0].twinx() )

# select nodes by device id
nodes = [ n for n in db if n['device'] == '1' ]

# sort by proc time
#nodes = [ nodes[j] for j in np.argsort([nodes[i]['proc_time'] for i in range(len(nodes))]) ]


labels = [
    'full',
    'half',
    'dark',
    #'shunt',
    #'series'
]

#cnorm  = colors.Normalize(vmin = 0, vmax = len(nodes))
#smap = cmx.ScalarMappable(norm = cnorm, cmap = 'magma')

for i in range(len(nodes))[::2]:
    
    d = nodes[i]

    for l in labels:

        V = d[l]['voltage']
        I = -d[l]['current']
        #L = d[l]['intensity']

        j = np.argsort(V)
        V = V[j]
        I = I[j]
        #L = L[j]

        #lbl = '{}'.format(d['proc_time']) if l == labels[0] else None
        
        #ax[0].plot(V, I, '-', c = smap.to_rgba(i), label = lbl)
        ax[0].plot(V, I, '--', linewidth = 2, alpha = .8, label = l)
            
        #ax[1].plot(V, L, '--')#, c = smap.to_rgba(i))
    
ax[0].set_xlabel('Voltage (V)')
ax[0].set_ylabel('Current (A)')
#ax[1].set_ylabel('Illumination Intensity (mW)')

#ax[1].set_ylim(-.1, 1.1)

#ax[0].set_yscale('log')

ax[0].legend()
plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.close('all')

In [35]:
''' process measurement data '''

# process measurement data
db = pvlibs.process_file_data(db)


begin measurement data processing 

processing: measurement 1/14
processing: measurement 2/14
processing: measurement 3/14
processing: measurement 4/14
processing: measurement 5/14
processing: measurement 6/14
processing: measurement 7/14
processing: measurement 8/14
processing: measurement 9/14
processing: measurement 10/14
processing: measurement 11/14
processing: measurement 12/14
processing: measurement 13/14
processing: measurement 14/14

measurement data processing complete

14 measurements processed


In [37]:
''' compile and export data'''

# define list of measurement labels to export; format each as 'file_name_param': 'Output Label'
labels = {
    'wafer': 'Wafer Type',
    'device': 'Device',
    'side': 'Wafer Side'
}

# select values to export; format each as 'param': 'Output Label'
values = {
    'area': 'Area [cm^-2]',
    'isc': 'Isc [A]',
    'voc': 'Voc [V]',
    'pmpp': 'Pmpp [W]',
    'impp': 'Impp [I]',
    'vmpp': 'Vmpp [V]',
    'ff': 'FF []',
    'eta': 'Eta [%]',
    'rs': 'Rs [Ohm]',
    'rp': 'Rp [Ohm]',
    'jsc': 'Jsc [A/cm^-2]',
    'jmpp': 'Jmpp [A/cm^-2]',
    'rs_sqr': 'Rs [Ohm/cm^-2]',
    'rp_sqr': 'Rp [Ohm/cm^-2]',
}

# define results output file name
file_name = './iv-summary'

# compile dataset by selected labels and values
data = pvlibs.compile_data(db, labels, values, file_name)


                              Area [cm^-2]   Isc [A]   Voc [V]  Pmpp [W]  \
Wafer Type Device Wafer Side                                               
mono       12     back               0.243  8.639562  0.664129  4.585502   
                  front              0.243  9.643164  0.667489       NaN   
           13     back               0.243  8.662040  0.664126  4.617289   
                  front              0.243  9.650977  0.667392       NaN   
           14     back               0.243  8.512489  0.665773  4.554117   
                  front              0.243  9.649315  0.670016  5.146783   
           15     back               0.243  8.486231  0.664811  4.517730   
                  front              0.243  9.640096  0.669576       NaN   
           16     back               0.243  8.599027  0.662891  4.651885   
                  front              0.243  9.653764  0.666854       NaN   
multi      1      back               0.243  7.648853  0.645800  3.960132   
            